In [1]:
import sys
from pathlib import Path

try:
    notebook_path = Path(__file__).resolve()
    project_root = notebook_path.parents[4]
except NameError:
    project_root = Path.cwd().parents[3]

sys.path.insert(0, str(project_root))
print("Project root added to sys.path:", project_root)

# load data
from processor import Frames

Project root added to sys.path: /Users/ran/Seneca/BDC800-Capstone


/Users/ran/Seneca/BDC800-Capstone/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load data
frames_ds = Frames()
frames_ds.load()

In [12]:
# get list of documents
docs = list(frames_ds.get_documents())

# get list of question answer pairs
qas = list(frames_ds.get_queries())